In [7]:
# import requests

# req = requests.get('https://beomi.github.io/beomi.github.io_old/')

# html = req.text
# header = req.headers
# status = req.status_code
# is_ok = req.ok

In [5]:
# from selenium import webdriver
# from bs4 import BeautifulSoup
# import re
# driver = webdriver.Chrome('./chromedriver')
# driver.implicitly_wait(3)
# driver.get('https://www.brandi.co.kr/products/17184019')
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')
# img = soup.select('img[data-v-e6eb87de]')
# img_only = list(set(img))
# print()
# for n in img_only:
#     print (n['src'])


https://image.brandi.me/cproduct/2020/06/08/17184019_1591610307_image3_L.jpg
https://image.brandi.me/cproduct/2020/06/08/17184019_1591610306_image2_L.jpg
https://image.brandi.me/cproduct/2020/06/08/17184019_1591610306_image1_L.jpg


In [4]:
# print(img_only)

['<img data-v-e6eb87de="" src="https://image.brandi.me/cproduct/2020/06/08/17184019_1591610307_image3_L.jpg" style="width: 100%; position: absolute;"/>', '<img data-v-e6eb87de="" src="https://image.brandi.me/cproduct/2020/06/08/17184019_1591610306_image2_L.jpg" style="width: 100%; position: absolute;"/>', '<img data-v-e6eb87de="" src="https://image.brandi.me/cproduct/2020/06/08/17184019_1591610306_image1_L.jpg" style="width: 100%; position: absolute;"/>']


In [62]:
# import re
# a = 'src="https://image.brandi.me/cproduct/2020/05/29/16841528_1590732351_image1_L.jpg"'
# regex = re.findall(r'src=\w*(\d*)\w?[.]jpg',a)
# print(regex)

[]


In [78]:
# a= 'src=https://image.brandi.me/cproduct/2020/05/29/16841528_1590732351_image1_L.jpg'
# regex = re.search(r'(.).{2}[.]jpg',a)
# print(regex.start())

73


In [6]:
# regex = sorted(img_only, key=lambda img_only: img_only[1])

In [98]:
# b = ['src=img3as.jpg asdff','src=img2as.jpg asdf','src=img1as.jpg asdfsf']
# # so = sorted(b, key=lambda b:b[re.findall(r'(.).{2}[.]jpg')])
# s = sorted(b, key=lambda b:b[re.search(r'(.).{2}[.]jpg',b).start()])
# print(s)
# # for n in b:
# #     re.search
# # key=re.findall(r'(.).{2}[.]jpg'

['src=img1as.jpg asdfsf', 'src=img2as.jpg asdf', 'src=img3as.jpg asdff']


In [83]:
# print(so)

['src=img3.jpg asdf', 'src=img2.jpg asdfasf', 'src=img1.jpg asdf']


In [89]:
# c= ['c3','b2','a1']
# print(sorted(c, key=lambda c:c[1]))

['a1', 'b2', 'c3']


In [108]:
# b = img_only
# d = list(map(str,b))
# regex = sorted(d, key=lambda d:d[re.search(r'(.).{2}[.]jpg',d).start()])
# print(regex)

# # s = sorted(b, key=lambda b:b[re.search(r'(.).{2}[.]jpg',b).start()])
# # print(s)           

['<img data-v-e6eb87de="" src="https://image.brandi.me/cproduct/2020/05/29/16841528_1590732351_image1_L.jpg" style="width: 100%; position: absolute;"/>', '<img data-v-e6eb87de="" src="https://image.brandi.me/cproduct/2020/05/29/16841528_1590732352_image2_L.jpg" style="width: 100%; position: absolute;"/>', '<img data-v-e6eb87de="" src="https://image.brandi.me/cproduct/2020/05/29/16841528_1590732353_image3_L.jpg" style="width: 100%; position: absolute;"/>', '<img data-v-e6eb87de="" src="https://image.brandi.me/cproduct/2020/05/29/16841528_1590732356_image4_L.jpg" style="width: 100%; position: absolute;"/>', '<img data-v-e6eb87de="" src="https://image.brandi.me/cproduct/2020/05/29/16841528_1590732356_image5_L.jpg" style="width: 100%; position: absolute;"/>']


In [74]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(3)
driver.get('https://www.brandi.co.kr/products/18306722')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
img = list(set(soup.select('img[data-v-e6eb87de]')))
for n in img:
    print (n['src'])

https://image.brandi.me/cproduct/2020/07/16/18306722_1594888395_image1_L.jpg


In [75]:
brandname = soup.select('a[data-v-c8060bfa]')[1].get_text()
print(brandname)

유니온스포츠


In [76]:
title = soup.select_one('.detail_title').get_text()
print(title)

[나이키] NSW ESSNTL TOP SS LBR OS_BV3929-010


In [77]:
try:
    data = soup.select_one('.detail_price b').get_text()
    discountRate = int(re.findall(r'\d*',data)[0].replace('%',''))
except:
    discountRate = 0

print(discountRate)

72


In [78]:
try:
    data = soup.select_one('.detail_price span').get_text()
except:
    data = soup.select_one('.detail_price').get_text()
actualPrice = int(re.findall(r'\d*[,]\d*',data)[0].replace(',',''))
print(actualPrice)

69000


In [79]:
data = soup.select_one('.detail_purchase_count').get_text()
try:
    Purchase_count = int(re.findall(r'\d*[,]\d*',data)[0].replace(',',''))
except:
    Purchase_count = int(re.findall(r'\d*',data)[0])
print(Purchase_count)

259


In [80]:
delivery = soup.select_one('.detail_info_wrapper > span').get_text()[5:]
print(delivery)

판매자 배송


In [81]:
option = soup.select('.detail_options option')
for i in option:
    print(i['value'])


S
M
L
XL
2XL


In [82]:
info_img = soup.select('#info_container_1 img')
for i in info_img:
    print(i['src'])

http://gi.esmplus.com/unisp123/images/0703_/BV3929-010.jpg


In [83]:
info_text = [i for i in soup.select_one('#info_container_1').get_text().splitlines() if i]
print(info_text)
while '\xa0' in info_text:
    info_text.remove('\xa0')
for i in info_text:
    print(i)

[]
